In [1]:
import pathlib
import textwrap
import os
import google.generativeai as genai
from IPython.display import display, Markdown, HTML
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
import json
import re
import tiktoken

# Load environment variables
load_dotenv(find_dotenv())
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

# Configure the API client with the API key
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-1.5-pro')


In [2]:
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def upload_rubric(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

# Documentation for Grading Functions

## Function: format_prompt

### Description
The `format_prompt` function constructs a prompt string that includes the grading rubric, the student's essay, and any additional information. This prompt is used to validate the rubric and grade the essay if the rubric is appropriate.

### Parameters
- **rubric_content** (`str`): The content of the grading rubric in string format.
- **essay_text** (`str`): The text of the essay to be graded.
- **additional_info** (`str`): Extra information to consider during grading.

### Returns
- **prompt** (`str`): A formatted string that includes the grading rubric, the student's essay, and the additional information, with specific instructions for validation and grading.

## Function: analyze_essay_with_rubric

### Description
The `analyze_essay_with_rubric` function takes a rubric content, an essay text, additional information, and a model to generate grading feedback for the essay based on the rubric. The function constructs a prompt using the rubric content, essay text, and additional information, and then uses the provided AI model to generate the grading feedback.

### Parameters
- **model** (`GenerativeModel`): The AI model used to generate the content.
- **rubric_content** (`str`): The content of the grading rubric in string format.
- **essay_text** (`str`): The text of the essay to be graded.
- **additional_info** (`str`): Extra information to consider during grading.

### Returns
- **response.text** (`str`): The grading feedback generated by the model.

### Usage
1. The function starts by counting tokens for each component (rubric, essay, and additional info) using the provided model's `count_tokens` method.
2. It constructs a prompt using the `format_prompt` function.
3. The total tokens for the entire prompt are counted.
4. The function generates a response using the model's `generate_content` method.
5. The response's token usage is printed, and the grading feedback is returned as a string.

## Function: get_additional_info

### Description
The `get_additional_info` function allows the user to select or enter additional information that should be considered during grading. This additional information can influence the grading criteria and feedback provided by the model.

### Parameters
- None

### Returns
- **additional_info** (`str`): A string containing the selected or entered additional information.

### Usage
1. The function presents a list of preset options for additional information to the user.
2. The user can choose one of these options or enter their own custom additional information.
3. The selected or entered information is then returned as a string.

#### Preset Options:
1. Be extra harsh when grading
2. Add additional encouragement for the student
3. Focus on grammar and syntax
4. Consider creativity and originality
5. Suggest reading materials for the student based on the grading
6. Enter your own additional information


In [3]:
def format_prompt(rubric_content, essay_text):
    return f"""
    1. Evaluate the subject of the following student essay and the provided rubric to ensure they match:

    Grading Rubric:
    {rubric_content}

    Student Essay:
    {essay_text}
    
    1a. Determine the subject of the essay.
    1b. Examine if there are any anomalies in the essay or rubric that would require teacher intervention. If yes, please notify the user why and DO NOT grade the essay. Examples include, but are not limited to:
        - The rubric is not appropriate for the subject matter
        - Irrelevant subject matter
        - Essays with only a few sentences
        - Essays that significantly deviate from the expected structure or content
        - The essay is written in a language other than the expected one
        - The essay contains inappropriate or offensive content
        - The rubric does not cover the key elements discussed in the essay
    1c. If there are no anomalies, please move to step 2 and grade the essay with the rubric.

    2. Please ensure the following when grading:
    2a. Always grade the essay, giving a score as specified in the rubric that is in the output.
    2b. The grading scores should be logical and consistent with the rubric criteria.
    2c. Provide explanations for each criterion.
    2d. Calculate the final score correctly. If weights are specified in the rubric, calculate the final score as a weighted average. If no weights are specified, sum the scores for each criterion.

    Sample Output Format:
    
    ### Appropriateness Analysis 
    
    Essay Subject: Note the subject of the essay as found in 1a. Reference parts of the student assignment and provide a high-level overview of any issues with the assignment.
    Essay Appropriateness: For the essay, if there were anomalies as found in 1b, please state why and why you did not continue grading.
    Rubric Appropriateness: For the rubric, if there were anomalies as found in 1b, please state why and why you did not continue grading.

    ### Grading Rubric and Feedback

    [Criterion Name]
    Score: [Score]
    Explanation: [Explanation]

    [Criterion Name]
    Score: [Score]
    Explanation: [Explanation]

    Total Score: [Total Score]

    Note: Do not include the word "Criterion" or criterion numbers next to the actual criterion names in the feedback. Always provide an explanation and overall feedback.
    """


def analyze_essay_with_rubric(model, rubric_content, essay_text, additional_info):
    # Count input tokens for each component
    rubric_tokens = model.count_tokens(rubric_content).total_tokens
    essay_tokens = model.count_tokens(essay_text).total_tokens
    additional_info_tokens = model.count_tokens(additional_info).total_tokens

    print(f"Rubric Token Count: {rubric_tokens}")
    print(f"Essay Token Count: {essay_tokens}")
    print(f"Additional Info Token Count: {additional_info_tokens}")
    
    prompt = format_prompt(rubric_content, essay_text, additional_info)
    
    # Count input tokens for the entire prompt
    prompt_tokens = model.count_tokens(prompt).total_tokens
    print(f"Grading Prompt Token Count: {prompt_tokens}")
    
    response = model.generate_content(prompt)
    
    # Ensure the response streaming completes
    response.resolve()
    
    # Print token usage details
    usage_metadata = response.usage_metadata
    print(f"Grading Response Token Count: {usage_metadata.candidates_token_count}")
    print(f"Grading Total Token Count: {usage_metadata.total_token_count}")
    print(" ")

    
    return response.text


def get_additional_info():
    preset_options = [
        "Be extra harsh when grading",
        "Add additional encouragement for the student",
        "Focus on grammar and syntax",
        "Consider creativity and originality",
        "Suggest reading materials for the student based on the grading"
    ]
    
    print("Select additional information for grading:")
    for i, option in enumerate(preset_options, 1):
        print(f"{i}. {option}")
    print("6. Enter your own additional information")

    choice = int(input("Enter the number of your choice: "))

    if 1 <= choice <= 5:
        additional_info = preset_options[choice - 1]
    elif choice == 6:
        additional_info = input("Enter your additional information: ")
    else:
        print("Invalid choice, please select a valid option.")
        return get_additional_info()

    return additional_info

# Functions for Essay Grading and Improvement Suggestions

## Function: format_suggestions_and_rewrite_prompt

### Description
The `format_suggestions_and_rewrite_prompt` function constructs a prompt string that includes the grading feedback and the student's essay. This prompt is used to generate suggestions for improvement and to rewrite the essay using the suggested improvements.

### Parameters
- **grading_feedback** (`str`): The feedback received from grading the essay, which includes the criterion, score, and explanation.
- **essay_text** (`str`): The text of the essay to be improved and rewritten.

### Returns
- **prompt** (`str`): A formatted string that includes the grading feedback and the student's essay, with specific instructions for generating suggestions and rewriting the essay. The instructions ask for the smallest chunk of the original text where each improvement was applied, the criterion from the rubric, and the reason for the suggested improvement, all formatted in JSON.

## Function: generate_suggestions_and_rewrite_essay

### Description
The `generate_suggestions_and_rewrite_essay` function takes grading feedback, an essay text, and a model to generate suggestions for improvement and rewrite the essay using those suggestions. The function constructs a prompt using the `format_suggestions_and_rewrite_prompt` function, which combines the grading feedback and essay text into a single prompt. It then uses the provided AI model to generate the suggestions and rewritten essay.

### Parameters
- **model** (`GenerativeModel`): The AI model used to generate the content.
- **grading_feedback** (`str`): The feedback received from grading the essay, which includes the criterion, score, and explanation.
- **essay_text** (`str`): The text of the essay to be improved and rewritten.

### Returns
- **response.text** (`str`): The suggestions and rewritten essay generated by the model. If the response from the model is empty or fails, the function returns `None`.

### Usage
1. **Construct Prompt**: The function calls the `format_suggestions_and_rewrite_prompt` function, passing in the grading feedback and essay text. This function returns a formatted prompt string.
2. **Count Tokens**: The function counts the input tokens for the grading feedback, essay text, and the entire prompt using the model's `count_tokens` method.
3. **Generate Content**: The function uses the AI model to generate content based on the constructed prompt by calling the model's `generate_content` method.
4. **Resolve Response**: The function ensures the response streaming completes by calling `response.resolve()`.
5. **Return Response**: If the model successfully generates a response, the function prints the response for debugging purposes and returns the generated text. If the response is empty or fails, the function prints an error message and returns `None`.


In [4]:
def format_suggestions_and_rewrite_prompt(grading_feedback, essay_text):
    return f"""
    Based on the following grading feedback,

    1. Provide suggestions for improvement.
    2. Rewrite the essay using the suggested improvements.

    Grading Feedback:
    {grading_feedback}

    Essay:
    {essay_text}

    For each improvement provided, identify and record the following:
    a. The smallest chunk of the original text where it was applied.
    b. The criterion from the rubric.
    c. The reason for the suggested improvement.

    Ensure that every change made in the rewritten essay is recorded in the following JSON format:
    [
        {{"improvement": "improvement_1", "criterion_from_rubric": "criterion_from_rubric_1", "reason_for_suggestion": "reason_for_suggestion_1", "original_text": "original_text_1", "revised_text": "revised_text_1"}},
        ...
        {{"improvement": "improvement_n", "criterion_from_rubric": "criterion_from_rubric_n", "reason_for_suggestion": "reason_for_suggestion_n", "original_text": "original_text_n", "revised_text": "revised_text_n"}}
    ]
    
    Note: never use characeters like ** or *


    """


def generate_suggestions_and_rewrite_essay(model, grading_feedback, essay_text):
    # Count input tokens for each component
    feedback_tokens = model.count_tokens(grading_feedback).total_tokens
    essay_tokens = model.count_tokens(essay_text).total_tokens

    print(f"Grading Feedback Token Count: {feedback_tokens}")
    print(f"Essay Token Count: {essay_tokens}")
    
    prompt = format_suggestions_and_rewrite_prompt(grading_feedback, essay_text)
    
    # Count input tokens for the entire prompt
    prompt_tokens = model.count_tokens(prompt).total_tokens
    print(f"Suggestions Prompt Token Count: {prompt_tokens}")
    
    response = model.generate_content(prompt)
    
    # Ensure the response streaming completes
    response.resolve()
    
    # Print token usage details
    usage_metadata = response.usage_metadata
    print(f"Suggestions Response Token Count: {usage_metadata.candidates_token_count}")
    print(f"Suggestions Total Token Count: {usage_metadata.total_token_count}")
    print(" ")
    
    if response and response.text:
        print("Full response from model:\n", response.text)  # Debugging statement
        return response.text
    else:
        print("Error: No response from model")
        return None


In [5]:
'''

def format_suggestions_and_rewrite_prompt(grading_feedback, essay_text):
    return f"""
    Based on the following grading feedback, 
    
    1. Provide suggestions for improvement
    2. Rewrite the essay using the suggested improvements

    Grading Feedback:
    {grading_feedback}

    Essay:
    {essay_text}
    
    Take each improvement provided and identify,
    a. the smallest chunk of the original text where it was applied
    b. the criterion from the rubric 
    c. the reason for the suggested improvement
    
    Provide steps a, b, and c in the following JSON format: 
    [
        {{"improvement": "improvement_1", "criterion_from_rubric": "criterion_from_rubric_1", "reason_for_suggestion": "reason_for_suggestion_1", "original_text": "original_text_1", "revised_text": "revised_text_1"}},
        ...
        {{"improvement": "improvement_n", "criterion_from_rubric": "criterion_from_rubric_n", "reason_for_suggestion": "reason_for_suggestion_n", "original_text": "original_text_n", "revised_text": "original_text_n"}}
    ]
    
    """


'''

'\n\ndef format_suggestions_and_rewrite_prompt(grading_feedback, essay_text):\n    return f"""\n    Based on the following grading feedback, \n    \n    1. Provide suggestions for improvement\n    2. Rewrite the essay using the suggested improvements\n\n    Grading Feedback:\n    {grading_feedback}\n\n    Essay:\n    {essay_text}\n    \n    Take each improvement provided and identify,\n    a. the smallest chunk of the original text where it was applied\n    b. the criterion from the rubric \n    c. the reason for the suggested improvement\n    \n    Provide steps a, b, and c in the following JSON format: \n    [\n        {{"improvement": "improvement_1", "criterion_from_rubric": "criterion_from_rubric_1", "reason_for_suggestion": "reason_for_suggestion_1", "original_text": "original_text_1", "revised_text": "revised_text_1"}},\n        ...\n        {{"improvement": "improvement_n", "criterion_from_rubric": "criterion_from_rubric_n", "reason_for_suggestion": "reason_for_suggestion_n", "

# Functions to Clean the JSON Output

In [6]:
def clean_json_string(json_string):
    # Remove Markdown formatting and trailing commas
    json_string = json_string.replace('```json', '').replace('```', '').strip()
    json_string = re.sub(r',\s*([}\]])', r'\1', json_string)  # Remove trailing commas before closing braces
    # Ensure all objects are closed properly
    json_string = re.sub(r'\s*}\s*{', '},{', json_string)
    # Add missing closing brackets if necessary
    if json_string.count('[') > json_string.count(']'):
        json_string += ']'
    if json_string.count('{') > json_string.count('}'):
        json_string += '}'
    return json_string

def extract_json_improvements(response):
    if response is None:
        print("Error: No response provided for JSON extraction")
        return {'improvements': [], 'response_without_json': ''}
    try:
        json_match = re.search(r'\[\s*\{.*\}\s*\]', response, re.DOTALL)
        if not json_match:
            raise ValueError("No valid JSON found in response.")
        
        cleaned_json_string = clean_json_string(json_match.group(0))
        print("Cleaned JSON string:", cleaned_json_string)  # Debugging statement

        improvements = json.loads(cleaned_json_string)
        response_without_json = response.replace(json_match.group(0), '')
        return {'improvements': improvements, 'response_without_json': response_without_json}
    except (ValueError, json.JSONDecodeError) as e:
        print("Error extracting JSON from response:", e)
        print(f"Failed to decode JSON string: {response}")  # Print the entire response for debugging
        return {'improvements': [], 'response_without_json': response}


# Function to Display Improvements

In [7]:
def display_improvements(improvements, original_text):
    if not improvements:
        print("No improvements to display.")
        return

    css = """
    <style>
    .improvement-tooltip {
        position: relative;
        display: inline-block;
        cursor: pointer;
        background-color: yellow;
        margin-bottom: 10px;
        padding: 5px;
        border: 1px solid black;
        border-radius: 4px;
    }
    .improvement-tooltip .tooltiptext {
        visibility: hidden;
        width: 300px;
        background-color: white;
        color: black;
        text-align: left;
        border: 1px solid #ddd;
        padding: 10px;
        border-radius: 6px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        position: absolute;
        z-index: 1;
        top: 100%;
        left: 50%;
        margin-left: -150px;
    }
    .improvement-tooltip:hover .tooltiptext {
        visibility: visible;
    }
    </style>
    """
    
    html_content = original_text
    
    for improvement in improvements:
        original = improvement.get('original_text', '')
        revised = improvement.get('revised_text', '')
        improvement_text = improvement['improvement']
        reason_for_suggestion = improvement['reason_for_suggestion']
        
        # Check if 'criterion_from_rubric' is present
        if 'criterion_from_rubric' in improvement:
            criterion = improvement['criterion_from_rubric']
            tooltip_html = f"""
            <div class="improvement-tooltip">
                {revised}
                <span class="tooltiptext">
                    <strong>Criterion:</strong> {criterion}<br>
                    <strong>Improvement:</strong> {improvement_text}<br>
                    <strong>Reason:</strong> {reason_for_suggestion}
                </span>
            </div>
            """
        else:
            tooltip_html = f"""
            <div class="improvement-tooltip">
                {revised}
                <span class="tooltiptext">
                    <strong>Improvement:</strong> {improvement_text}<br>
                    <strong>Reason:</strong> {reason_for_suggestion}
                </span>
            </div>
            """
        
        if original:
            html_content = html_content.replace(original, tooltip_html)
        else:
            html_content = html_content + tooltip_html
    
    display(HTML(css + html_content))


# Functions for Selecting and Listing Rubrics

In [9]:
def list_rubrics(rubric_folder):
    return [f for f in os.listdir(rubric_folder) if f.endswith('.md')]

def select_rubric(rubric_folder):
    rubrics = list_rubrics(rubric_folder)
    print("Available rubrics:")
    for idx, rubric in enumerate(rubrics, 1):
        print(f"{idx}. {rubric}")
    
    rubric_index = int(input("Select a rubric by entering the corresponding number: ")) - 1
    selected_rubric = rubrics[rubric_index]
    return os.path.join(rubric_folder, selected_rubric)

current_dir = os.getcwd()
relative_rubric_folder = '../rubrics'
rubric_folder = os.path.join(current_dir, relative_rubric_folder)

if not os.path.exists(rubric_folder):
    print(f"The directory {rubric_folder} does not exist. Creating the directory.")
    os.makedirs(rubric_folder)
    print(f"Please add rubric files in markdown format (.md) to the {rubric_folder} directory and re-run the notebook.")
else:
    rubric_files = list_rubrics(rubric_folder)
    if not rubric_files:
        print(f"No rubric files found in the directory {rubric_folder}. Please add rubric files in markdown format (.md) and re-run the notebook.")
    else:
        rubric_file_path = select_rubric(rubric_folder)
        if rubric_file_path:
            rubric_content = upload_rubric(rubric_file_path)
            display(to_markdown(rubric_content))

Available rubrics:
1. writing_ela_rubric_11_12_nys.md
2. writing_ela_rubric_9_10_nys.md
3. writing_social_sciences_11_12_nys.md
4. science_rubric_9_12_nys.md
Select a rubric by entering the corresponding number: 1


> # New York State Common Core 11-12th Grade English Language Arts Learning Standards Rubric
> 
> | Criterion                          | 1                                                                                  | 2                                                                                                | 3                                                                           | 4                                                                               | 5                                                                                     | 6                                                                                              | 7                                                                                                           |
> |---------|---------|---------|---------|---------|---------|---------|---------|
> | **1. Argument Development**        | No clear claim or counterclaim present; lacks significance and organization.       | Minimal claim with little distinction between counterclaims; weak organization and significance. | Claims and counterclaims present but lack depth and significance.           | Clear claims and counterclaims with logical organization and some significance. | Precise claims with clear distinction, logical sequence, and significance.            | Detailed claims and counterclaims, thoroughly balanced and significant.                        | Precise, knowledgeable claims with logical sequencing, strong distinction, and significant analysis.        |
> | **2. Evidence and Analysis**       | No relevant evidence provided; weak analysis and audience consideration.           | Minimal relevant evidence; weak development and limited audience consideration.                  | Some relevant evidence but lacks thorough analysis and audience engagement. | Relevant evidence with adequate analysis; some audience consideration.          | Strong evidence with thorough analysis; good audience consideration.                  | Comprehensive evidence and analysis; anticipates audience’s knowledge and biases.              | Most relevant evidence with detailed analysis; fully considers audience’s knowledge, biases, and concerns.  |
> | **3. Cohesion and Clarity**        | No transitions or clarity in relationships between ideas; lacks cohesion.          | Minimal use of transitions, unclear connections, weak cohesion.                                  | Basic use of transitions; some clarity and cohesion in connections.         | Appropriate transitions and clear relationships; good cohesion.                 | Effective use of varied transitions; clear and cohesive relationships.                | Skillful use of varied syntax and transitions; strong cohesion.                                | Highly effective use of transitions and syntax; excellent cohesion and clarity throughout.                  |
> | **4. Style and Tone**              | Inconsistent or inappropriate style and tone; does not adhere to discipline norms. | Minimal consistency in style and tone; weak adherence to discipline norms.                       | Basic consistency in style and tone; some adherence to discipline norms.    | Consistent style and tone appropriate to the task; adheres to discipline norms. | Formal style and objective tone maintained consistently; adheres to discipline norms. | Advanced formal style and objective tone; fully adheres to discipline norms.                   | Mastery of formal style and tone; fully appropriate to the writing task and discipline norms.               |
> | **5. Conclusion**                  | No conclusion provided; irrelevant to the argument.                                | Weak or irrelevant conclusion; minimal support to the argument.                                  | Basic conclusion; some relevance to the argument but lacks depth.           | Clear conclusion supporting the argument; some depth.                           | Strong conclusion effectively supporting the argument; good depth.                    | Compelling conclusion thoroughly supporting and reflecting on the argument; significant depth. | Insightful and compelling conclusion; fully supports and adds significant depth to the argument.            |
> | **6. Organization of Ideas**       | No clear organization of ideas; lacks connections and clarity.                     | Minimal organization with unclear connections; lacks clarity.                                    | Basic organization; some connections between ideas, but lacks clarity.      | Logical organization with clear connections between ideas; good clarity.        | Strong organization creating a cohesive narrative; clear and logical.                 | Advanced organization; building towards a clear outcome; very clear and logical.               | Mastery in organizing ideas; creating a unified and engaging narrative; highly clear and logical.           |
> | **7. Vocabulary and Language Use** | Inappropriate or minimal use of vocabulary; lacks complexity.                      | Basic vocabulary with limited complexity; weak usage.                                            | Some precise language but lacks consistency and complexity.                 | Appropriate vocabulary and precise language; some complexity.                   | Effective use of content-specific vocabulary and precise language; good complexity.   | Advanced use of vocabulary, literary techniques, and precise language; high complexity.        | Mastery of vocabulary, literary techniques, and language; fully appropriate to the complexity of the topic. |


In [10]:
# Get additional information from the user
additional_info = get_additional_info()

Select additional information for grading:
1. Be extra harsh when grading
2. Add additional encouragement for the student
3. Focus on grammar and syntax
4. Consider creativity and originality
5. Suggest reading materials for the student based on the grading
6. Enter your own additional information
Enter the number of your choice: 


ValueError: invalid literal for int() with base 10: ''

In [13]:
# US HISTORY ESSAY


# The essay text
essay_text =  """
The history of the United States is marked by a continuous struggle for civil rights and equality, shaping the nation's identity and societal structure. From the early days of colonization to the modern era, the quest for civil rights has been a central theme, reflecting the dynamic and often tumultuous nature of the American democratic experiment. This ongoing journey has significantly influenced the development of American society and democracy.
The roots of the civil rights movement can be traced back to the early colonial period, where notions of freedom and equality were paradoxically juxtaposed with the institution of slavery. The Declaration of Independence in 1776, with its proclamation that "all men are created equal," laid a philosophical foundation for civil rights, yet this ideal was not extended to all individuals, particularly African Americans and Native Americans. The contradiction between the ideals of the Declaration and the reality of slavery set the stage for future conflicts and movements aimed at achieving true equality.
The abolitionist movement of the 19th century was one of the earliest organized efforts to challenge slavery and advocate for the civil rights of African Americans. Figures such as Frederick Douglass, Harriet Tubman, and William Lloyd Garrison played pivotal roles in raising awareness and mobilizing support for the abolition of slavery. The publication of antislavery literature, such as Harriet Beecher Stowe's Uncle Tom's Cabin, also contributed to the growing sentiment against slavery in the North.
The culmination of the abolitionist movement was the Civil War (1861-1865), a conflict fundamentally rooted in the issue of slavery. The Union's victory and the subsequent passage of the 13th Amendment in 1865 abolished slavery, marking a significant milestone in the struggle for civil rights. However, the end of slavery did not mean the end of discrimination or the beginning of true equality. The Reconstruction era (1865-1877) saw significant advancements, including the 14th and 15th Amendments, which granted citizenship and voting rights to African Americans. Despite these legal gains, the period was also marked by severe backlash, including the rise of the Ku Klux Klan and the implementation of Jim Crow laws, which enforced racial segregation and disenfranchised Black citizens.
The early 20th century witnessed the emergence of new civil rights organizations and leaders who sought to challenge segregation and discrimination through legal means and grassroots activism. The National Association for the Advancement of Colored People (NAACP), founded in 1909, played a crucial role in challenging discriminatory laws and practices through the court system. One of the most significant victories was the Supreme Court's decision in Brown v. Board of Education (1954), which declared state laws establishing separate public schools for Black and white students to be unconstitutional.
The 1950s and 1960s are often regarded as the height of the civil rights movement, a period characterized by widespread activism, landmark legislation, and profound social change. The movement was marked by nonviolent protests, such as the Montgomery Bus Boycott (1955-1956) and the March on Washington (1963), where Dr. Martin Luther King Jr. delivered his iconic "I Have a Dream" speech. These actions, along with the courage and determination of countless individuals, brought national attention to the injustices faced by African Americans.
Key legislative achievements during this period included the Civil Rights Act of 1964 and the Voting Rights Act of 1965. The Civil Rights Act prohibited discrimination on the basis of race, color, religion, sex, or national origin and ended segregation in public places. The Voting Rights Act aimed to eliminate barriers to voting for African Americans, particularly in the South, where practices such as literacy tests and poll taxes had been used to disenfranchise Black voters.
Despite these significant advancements, the struggle for civil rights continued beyond the 1960s. The late 20th and early 21st centuries saw ongoing efforts to address issues such as racial profiling, police brutality, and economic inequality. The Black Lives Matter movement, which emerged in response to the killings of unarmed Black individuals by police, has brought renewed attention to the persistent racial disparities in American society.
The quest for civil rights in the United States has not been limited to African Americans. Other marginalized groups, including women, Native Americans, LGBTQ+ individuals, and immigrants, have also fought for recognition and equality. The women's suffrage movement, which culminated in the 19th Amendment in 1920, granted women the right to vote and marked a significant step toward gender equality. The LGBTQ+ rights movement achieved a major victory with the Supreme Court's decision in Obergefell v. Hodges (2015), which legalized same-sex marriage nationwide.
Throughout American history, the struggle for civil rights has been a dynamic and multifaceted endeavor. It has involved legal battles, grassroots activism, and significant sacrifices by individuals committed to the cause of equality. The progress achieved has been substantial, yet the journey is ongoing, with new challenges and opportunities for advancing civil rights continuing to emerge.
In conclusion, the history of civil rights in the United States is a testament to the resilience and determination of those who have fought for justice and equality. From the abolition of slavery to the civil rights movement of the 1960s and beyond, each generation has contributed to the ongoing effort to realize the ideals of freedom and equality upon which the nation was founded. As society continues to evolve, the legacy of these struggles serves as both a reminder of the progress made and a call to action to address the inequalities that persist today. The journey toward equality is far from over, but the commitment to civil rights remains a defining aspect of the American spirit. Each new movement and each legal victory brings the nation closer to the ideal of true equality, demonstrating that the pursuit of civil rights is an enduring and essential part of the American story. The impact of these movements reverberates through history, showcasing the power of collective action and the relentless pursuit of justice. As new challenges arise, the foundational values of equality and justice continue to guide and inspire future generations to push forward.
"""





In [14]:
# Biology Lab Report

'''

essay_text = """
The Process of Photosynthesis
Introduction
Photosynthesis is a fundamental process that takes place in the chloroplasts of plant cells, allowing plants to convert light energy into chemical energy. This process is crucial for the survival of plants and other autotrophic organisms, as it forms the basis of the food chain. In this report, we will explore the mechanisms of photosynthesis, the factors that affect it, and its importance to life on Earth.
The Process of Photosynthesis
Photosynthesis occurs in two main stages: the light-dependent reactions and the Calvin cycle (light-independent reactions).
Light-Dependent Reactions
These reactions take place in the thylakoid membranes of the chloroplasts. During the light-dependent reactions, chlorophyll and other pigments absorb light energy, which is then used to split water molecules (H₂O) into oxygen (O₂), protons, and electrons. This process, known as photolysis, releases oxygen as a by-product. The energy captured from light is stored in the form of ATP (adenosine triphosphate) and NADPH (nicotinamide adenine dinucleotide phosphate).
The overall equation for the light-dependent reactions can be summarized as: 2H2O+2NADP++3ADP+3Pi+lightenergy→2NADPH+3ATP+O22H2O+2NADP++3ADP+3Pi+lightenergy→2NADPH+3ATP+O2
Calvin Cycle (Light-Independent Reactions)
The Calvin cycle takes place in the stroma of the chloroplasts. During this cycle, the ATP and NADPH produced in the light-dependent reactions are used to convert carbon dioxide (CO₂) into glucose (C₆H₁₂O₆). This process involves a series of enzyme-mediated steps, starting with the fixation of CO₂ into a five-carbon sugar, ribulose bisphosphate (RuBP), and ending with the production of glucose.
The overall equation for the Calvin cycle can be summarized as: 6CO2+18ATP+12NADPH+H2O→C6H12O6+18ADP+18Pi+12NADP+6CO2+18ATP+12NADPH+H2O→C6H12O6+18ADP+18Pi+12NADP+
Factors Affecting Photosynthesis
Several factors can influence the rate of photosynthesis, including:
•	Light Intensity: Higher light intensity increases the rate of photosynthesis up to a certain point, beyond which the rate will plateau.
•	Carbon Dioxide Concentration: An increase in CO₂ concentration can enhance the rate of photosynthesis, as CO₂ is a key substrate for the Calvin cycle.
•	Temperature: Photosynthesis is temperature-dependent, with the rate increasing up to an optimal temperature range, after which it declines due to enzyme denaturation.
•	Water Availability: Water is essential for photolysis in the light-dependent reactions. A shortage of water can limit photosynthesis.
Importance of Photosynthesis
Photosynthesis is vital for life on Earth for several reasons:
1.	Oxygen Production: Photosynthesis is the primary source of atmospheric oxygen, which is essential for the respiration of most living organisms.
2.	Food Supply: It produces glucose, which is a fundamental energy source for plants and, indirectly, for animals and humans through the food chain.
3.	Carbon Dioxide Regulation: Photosynthesis helps regulate the levels of CO₂ in the atmosphere, contributing to the mitigation of climate change.
Conclusion
In summary, photosynthesis is a complex but essential process that converts light energy into chemical energy, supporting life on Earth. By understanding the mechanisms and factors that affect photosynthesis, we can better appreciate its role in maintaining ecological balance and sustaining the food supply.


"""

'''



'\n\nessay_text = """\nThe Process of Photosynthesis\nIntroduction\nPhotosynthesis is a fundamental process that takes place in the chloroplasts of plant cells, allowing plants to convert light energy into chemical energy. This process is crucial for the survival of plants and other autotrophic organisms, as it forms the basis of the food chain. In this report, we will explore the mechanisms of photosynthesis, the factors that affect it, and its importance to life on Earth.\nThe Process of Photosynthesis\nPhotosynthesis occurs in two main stages: the light-dependent reactions and the Calvin cycle (light-independent reactions).\nLight-Dependent Reactions\nThese reactions take place in the thylakoid membranes of the chloroplasts. During the light-dependent reactions, chlorophyll and other pigments absorb light energy, which is then used to split water molecules (H₂O) into oxygen (O₂), protons, and electrons. This process, known as photolysis, releases oxygen as a by-product. The energy c

In [15]:
# Validate and grade the essay
grading_feedback = analyze_essay_with_rubric(model, rubric_content, essay_text, additional_info)
print("Grading Feedback:")
print(grading_feedback)
display(to_markdown(grading_feedback))


NameError: name 'additional_info' is not defined

In [16]:
# Generate suggestions and rewrite the essay based on the grading feedback
suggestions_and_rewrite = generate_suggestions_and_rewrite_essay(model, grading_feedback, essay_text)
print(suggestions_and_rewrite)

NameError: name 'grading_feedback' is not defined

In [17]:
extracted_data = extract_json_improvements(suggestions_and_rewrite)
print("Extracted Data:", extracted_data)  # Debugging statement


NameError: name 'suggestions_and_rewrite' is not defined

In [18]:
# Display the suggestions and revised essay
display(Markdown(f"**Suggestions and Revised Essay:**\n\n{extracted_data['response_without_json']}"))


NameError: name 'extracted_data' is not defined

In [ ]:
# Display the improvements in the new essay with hover effect
display_improvements(extracted_data['improvements'], essay_text)
